# Recommendation system using collaborative filtering and the MovieLens dataset

This movie dataset contains 100,836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018.  This dataset was generated on September 26, 20.

Obtained from: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic informationis included.  Each user is represented by an id, and no other information is provided. The data are contained in the  files links.csv, movies.csv, ratings.csv and tags.csv

In [18]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [19]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/ratings.csv')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('data/movies.csv')

tnames = ['user_id', 'movie_id', 'tags', 'timestamp']
tags = pd.read_csv('data/tags.csv')

In [20]:
ratings[:5]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [21]:
movies[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
tags[:5]

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## Data Pre-Processing

Merging ratings and movies tables

In [53]:
movie_ratings = pd.merge(ratings,movies)
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Create unique integer encoding for each user and each movie

In [54]:

user_ids = movie_ratings["userId"].unique().tolist()
# use this dictionary to get unique integer encoding from original userid
user_encoding = {x: i for i, x in enumerate(user_ids)}
# use this dictionary to get original userid from integer encoding value
user_id_lookup = {i: x for i, x in enumerate(user_ids)}

# Same process for movies
movie_ids = movie_ratings["movieId"].unique().tolist()
# use this dict to get unique integer encoding from original movie id
movie_encoding = {x: i for i, x in enumerate(movie_ids)}
# use this dict to get original movieid from integer encoding value
movie_id_lookup = {i: x for i, x in enumerate(movie_ids)}

# Add new columns for unique integer encodings
movie_ratings["user_encoding"] = movie_ratings["userId"].map(user_encoding)
movie_ratings["movie_encoding"] = movie_ratings["movieId"].map(movie_encoding)


movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres,user_encoding,movie_encoding
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,0
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,0
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,0
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,0


Converting table to a ratings matrix using user_encoding as unique rows and movie_encoding as unique columns. We do this so we can use the index of the np array to represent a particular user-item pair. ie. ratings_matrix[2,4] represents user 2's movie rating for movie 4. This would not work if we used the default userid and movieid since there are missing ids in this dataset. 

In [110]:
user_ratings = movie_ratings.pivot_table('rating', index='user_encoding', columns='movie_encoding')
user_ratings = user_ratings.fillna(0)

item_ratings = movie_ratings.pivot_table('rating', index='movie_encoding', columns='user_encoding')
item_ratings = item_ratings.fillna(0)

ratings_matrix = user_ratings.to_numpy()
#ratings_matrix

ratings_matrix_items = item_ratings.to_numpy()
ratings_matrix

array([[4. , 4. , 4. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

Now that we have the proper format for our data matrix, we will split the data into training and testing sets. We will take 15 ratings from each user and use it for the test set

In [116]:
def train_test_split(ratings):
    # initialize test array with all 0s. Will be replaced by randomly selected ratings
    train = ratings.copy()
    test = np.zeros(ratings.shape)

    # go through each user
    for i in range(len(ratings_matrix)):
        
        # Take 15 random ratings that are not 0 and place them in the test set
        test_values = np.random.choice(ratings[i].nonzero()[0], size=15)
        
        # remove 15 ratings from training set
        train[i, test_values] = 0.
        
        # Add 15 rating to test set
        test[i, test_values] = ratings[i, test_values]
        
    
    return train, test

train, test = train_test_split(ratings_matrix)
item_train, item_test = train_test_split(ratings_matrix_items)

Next we will calculate similarity between users using cosine similary measure

In [117]:
user_sim = cosine_similarity(train, dense_output=False)
print(user_sim)

[[1.         0.10318585 0.16280841 ... 0.         0.         0.        ]
 [0.10318585 1.         0.08601394 ... 0.05165088 0.         0.        ]
 [0.16280841 0.08601394 1.         ... 0.0669029  0.02761129 0.        ]
 ...
 [0.         0.05165088 0.0669029  ... 1.         0.         0.        ]
 [0.         0.         0.02761129 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [118]:
item_sim = cosine_similarity(train.T, dense_output=False)
print(item_sim)

[[1.         0.27325176 0.35883836 ... 0.09105973 0.09105973 0.09105973]
 [0.27325176 1.         0.23055021 ... 0.         0.         0.        ]
 [0.35883836 0.23055021 1.         ... 0.12954281 0.12954281 0.12954281]
 ...
 [0.09105973 0.         0.12954281 ... 1.         1.         1.        ]
 [0.09105973 0.         0.12954281 ... 1.         1.         1.        ]
 [0.09105973 0.         0.12954281 ... 1.         1.         1.        ]]


In [132]:
print(str(len(user_sim[0])) + " " + str(len(train[0])))
print(str(len(item_sim[0])) + " " + str(len(item_train[0])))

610 9724
9724 610


In [106]:
user_sim[0]

array([1.        , 0.10785396, 0.14037775, 0.12481003, 0.19263516,
       0.19644447, 0.30075766, 0.13210956, 0.20865584, 0.12904635,
       0.15719837, 0.14136494, 0.08104609, 0.12789549, 0.10665548,
       0.30349463, 0.09120269, 0.09958916, 0.10434783, 0.33995237,
       0.19041719, 0.3021994 , 0.22248137, 0.25110321, 0.0855189 ,
       0.03891894, 0.15651848, 0.15472952, 0.20401106, 0.08524163,
       0.02607592, 0.05246066, 0.32840643, 0.21979059, 0.23935678,
       0.0421641 , 0.19222475, 0.0446147 , 0.16250538, 0.05145366,
       0.09162253, 0.16686502, 0.06776309, 0.12799247, 0.13368339,
       0.3019355 , 0.19257831, 0.19477801, 0.15790579, 0.17720952,
       0.06299408, 0.0692695 , 0.06073847, 0.15452329, 0.19419181,
       0.04753497, 0.26952624, 0.04589535, 0.21238966, 0.19209247,
       0.15266341, 0.20587899, 0.19175726, 0.14033028, 0.11594259,
       0.24622945, 0.02728069, 0.21438503, 0.08427572, 0.07925287,
       0.15795318, 0.20087183, 0.29887381, 0.06733294, 0.05032

In [129]:
print(len(item_sim[0]))
print(len(item_train[0,:]))

9724
610


Use the similarities to make predictions on unknown ratings

In [131]:
def make_ratings_predictions(ratings, similarity, based = 'user'):
    # set default array with all 0s. Will be replaced by predicted ratings
    preds = np.zeros(ratings.shape)
    
    # number of users/items
    n = len(similarity)
    # number of ratings 
    m = len(ratings[0])
    
    if based == 'user':
        for i in range(n):
            for j in range(m):
                # For each prediction we take the weighted sum of ratings using the similarity between users as the weights.
                # Using numpy dot method we can multiply the entire row in the user similarity matrix by the entire column in
                # the ratings matrix and add all those values up. Then we divide by the sum of the same row in the
                # user similarities matrix (ie. sum of similarity weights for respective user).
                weighted_sum = similarity[i].dot(ratings[:, j]) / np.sum(similarity[i])
                preds[i,j] = weighted_sum
    elif based == 'item':
        for i in range(n):
            for j in range(m):
                weighted_sum = similarity[j].dot(ratings[:, i].T) / np.sum(similarity[j])
                preds[i,j] = weighted_sum
        
    return preds

#predictions_matrix = make_ratings_predictions(train, user_sim)
predictions_matrix = make_ratings_predictions(item_train, item_sim, 'item')

# Compare all nonzero values in prediction matrix with all nonzero values in test set
#predictions = predictions_matrix[test.nonzero()]
#actual_ratings = test[test.nonzero()]
#mse = mean_squared_error(predictions, actual_ratings)


# Compare all nonzero values in prediction matrix with all nonzero values in test set
predictions = predictions_matrix[item_test.nonzero()]
actual_ratings = item_test[item_test.nonzero()]
mse = mean_squared_error(predictions, actual_ratings)

print("Mean Squared Error for user-based collaborative filtering: {:.3f}".format(mse))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


IndexError: index 610 is out of bounds for axis 1 with size 610

Currently, we are taking into account similarities between all users despite most being not very similar at all. We can try using only the top k most similar users to see if we can get better results.

In [89]:
# Same method but only using top k similar users
def make_ratings_predictions_top_k(ratings, user_similarity, k):
    preds = np.zeros(ratings.shape)
    
    # number of users
    n = len(user_similarity)
    # number of ratings 
    m = len(ratings[0])
    
    inverse_k = n-k+1
    
    for i in range(n):
        
        # Get k most similar users to user i from user_similarity matrix
        k_users = np.argsort(user_similarity[:,i])[:inverse_k:-1]
        
        for j in range(m):
            
            # index only k most similar users from similarit matrix
            weighted_sum = user_similarity[i][k_users].dot(ratings[:, j][k_users]) / np.sum(user_similarity[i][k_users])
            preds[i,j] = weighted_sum
    return preds


k = 30
predictions_matrix = make_ratings_predictions_top_k(train, user_sim, k)

# Compare all nonzero values in prediction matrix with all nonzero values in test set
predictions = predictions_matrix[test.nonzero()]
actual_ratings = test[test.nonzero()]
mse = mean_squared_error(predictions, actual_ratings)

print("Mean Squared Error for user-based collaborative filtering using top {} most similar users: {:.3f}".format(k, mse))

Mean Squared Error for user-based collaborative filtering using top 30 most similar users: 8.306


A function to request a specific rating on a movie, given the movie ID and user ID.

In [95]:
def get_movie_prediction(userid, movieid, predictions):
    # Use user lookup dict we made before
    u_encoded = user_id_lookup[userid]
    
    # Use movie lookup dict we made before
    m_encoded = movie_id_lookup[movieid]
    
    # Grab prediction from np array    
    return predictions[u_encoded, m_encoded]

mid = 54
uid = 19
p = get_movie_prediction(uid, mid, predictions_matrix)

print("Predicted rating for user {} on movie {} is: {}".format(uid,mid,p))

Predicted rating for user 19 on movie 54 is: 0.735567580568081
